In [40]:
import pandas as pd
import seaborn as sns

In [41]:
datadir = '../data/'

In [42]:
def _read_file(fname, enc='iso8859-8'):
    fd = open(fname, encoding=enc, errors='replace')
    df = pd.read_csv(fd, sep='|')
    return df

def add_model(df):
    df['model'] = df.apply(lambda x: '_'.join([x[y] for y in ['tozeret_cd', 'degem_cd', 'shnat_yitzur', 'sug_degem']]), axis=1)

def get_model_name(ns):
    names = pd.DataFrame(data={'model':ns})
    ret = pd.merge(names, models, how='left', on='model')[['tozeret_nm', 'kinuy_mishari']]
    return ret

In [43]:
# source : https://data.gov.il/dataset/private-and-commercial-vehicles/resource/053cea08-09bc-40ec-8f7a-156f0677aff3?inner_span=True

#df = pd.read_csv('data/rechev-small.csv', encoding='iso-8859-1', sep='|')

def read_file(fname):
    fname = datadir + fname
    df = _read_file(fname)

    for c in ['mispar_rechev', 'degem_cd', 'tozeret_cd', 'shnat_yitzur']:
        df[c] = df[c].astype(str)

    df['test']= pd.to_datetime(df.mivchan_acharon_dt)
    df['test_expiry']= pd.to_datetime(df.tokef_dt)    
    add_model(df)
    return df

In [44]:
prev = read_file('rechev.csv')
curr = read_file('rechev-20230413.csv')

In [65]:
# perform left and right joins
df_prev = pd.merge(prev, curr['mispar_rechev'], on='mispar_rechev', how='left')
df_prev['src'] = 'prev'
df_curr = pd.merge(prev['mispar_rechev'], curr, on='mispar_rechev', how='right')
df_curr['src'] = 'curr'
df = pd.concat([df_prev, df_curr])
df.drop_duplicates(subset='mispar_rechev', keep=False, inplace=True)

In [67]:
df['src'].value_counts()

curr    6182
prev    5718
Name: src, dtype: int64

In [74]:
df.query('src == "prev"')['shnat_yitzur'].value_counts()

2008    440
2019    349
2011    334
2007    327
2021    311
2018    298
2010    294
2016    276
2013    263
2015    259
2017    252
2012    252
2006    247
2003    223
2005    220
2014    215
2009    197
2002    174
2004    167
2001    149
2020    139
2022    117
2000     76
1999     60
1998     41
1997     21
2023     15
1996      2
Name: shnat_yitzur, dtype: int64

In [75]:
df.query('src == "curr"')['shnat_yitzur'].value_counts()

2023    4866
2017      90
2016      90
2019      84
2021      84
2008      81
2018      80
2020      79
2011      78
2015      78
2012      67
2010      64
2013      56
2003      54
2014      50
2009      49
2002      47
2007      42
2006      35
2005      26
2001      19
2004      17
2000      14
2022      11
1999       9
1997       7
1998       4
1996       1
Name: shnat_yitzur, dtype: int64